In [28]:
%pip install scikit-learn


   ---------------------------------------- 10.6/10.6 MB 851.3 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\danie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import pipeline #hugging face pipeline
from catboost import CatBoostClassifier
import ast
import os



Nesta estratégia eu classifiquei todos os reviews usando a análise de sentimento e somei os valores (positivo = 1, negativo = -1, neutro = 0) para cada local.

O resultado foi agregado no exported_data.csv, que contém o nome do local e a soma dos valores de sentimento.

In [42]:
x_train = pd.read_csv('./kaggle/input/desafioutfpr2024/X_trainToronto.csv')


                  business_id                         name  \
15     StWSgLnWkuSocMbMcwEQkw  Chef Jenny G's Fish & Chips   
22     HiUSB5_OqtN_DKJST3JZmw                Toro Wine bar   
31     TJresValKB2aHcmrwNEskQ              Djerba la Douce   
98     JOYXbgWP_aedNdASBRLC_g         Girl Friday Clothing   
103    Erf_MwkL0Gls3tCJvz41-A                  180 Secondi   
...                       ...                          ...   
17381  otsjAjxf0PNQ99xcmuj_LA    Sushi Making For the Soul   
17421  4mjGfztbrud_3krnjRPrig      Canadian Blood Services   
17506  4Jm1T9TlW5CzY9FVCyNNTw  Health Valley Spa and Salon   
17514  CMdfStxi0thj-ZLa8qD3hQ               Kwame's Fusion   
17568  VR0D2Le_OGEJTnYHCSrFnA                Comics & More   

                                     address postal_code   latitude  \
15                       2222 Queen Street E     M4E 1H2  43.672546   
22                      1588 Dundas Street W     M6K 1T8  43.650012   
31                         1475 Danforth A

In [ ]:
Aqui eu faço uma engenharia de features, transformando a coluna 'attributes' em uma soma do número de atributos "true". Ou seja, somei todos os atributos true para cada local
a ideia era que lugares com mais atributos "true" fossem mais propensos a serem visitados.
utilizei também a coluna review_count, que é o número de reviews que o local tem, como uma feature, pois indicaria a popularidade do local.
Por fim utilizei o parametro is_open, apra entender a relação com a popularidade do local.

In [44]:

x_train = pd.read_csv('./kaggle/input/desafioutfpr2024/X_trainToronto.csv')
x_scores = pd.read_csv('exported_review_sum.csv')


merged_df = pd.merge(x_train[['business_id', 'review_count', 'attributes', 'is_open', 'destaque']], x_scores,on='business_id')

attributes_df = pd.json_normalize(merged_df['attributes'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else {}))

merged_df['label_divided'] = merged_df['label'] / merged_df['review_count']
x_train = merged_df[['review_count', 'label_divided', 'is_open']]
y_train = merged_df['destaque']
x_train['price'] = attributes_df['RestaurantsPriceRange2'].replace('None', np.nan).astype(float)
x_train['attributes'] = attributes_df.loc[:, attributes_df.eq('True').any()].count(axis=1)
average_price = x_train['price'].mean()
x_train['price'].fillna(average_price, inplace=True)

x_train


C:\Users\danie\AppData\Local\Temp\ipykernel_7060\1551903115.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train['price'] = attributes_df['RestaurantsPriceRange2'].replace('None', np.nan).astype(float)
C:\Users\danie\AppData\Local\Temp\ipykernel_7060\1551903115.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train['attributes'] = attributes_df.loc[:, attributes_df.eq('True').any()].count(axis=1)
C:\Users\danie\AppData\Local\Temp\ipykernel_7060\1551903115.py:15: FutureWarning: A value is tryin

,review_count,label_divided,is_open,price,attributes
0,99,0.449040,1,3.000000,9
1,3,1.331739,1,2.000000,2
2,18,0.221720,1,1.948135,2
3,3,0.338114,0,1.948135,1
4,3,-0.999251,1,3.000000,0
...,...,...,...,...,...
17577,16,-0.251374,1,1.000000,12
17578,4,0.998909,1,2.000000,2
17579,8,-0.125223,1,2.000000,1
17580,9,-0.776320,1,2.000000,9


In [45]:

x_test = pd.read_csv('./kaggle/input/desafioutfpr2024/X_testToronto.csv')
x_test_scores = pd.read_csv('exported_review_sum_test.csv')

merged_df = pd.merge(x_test[['business_id', 'review_count','is_open', 'attributes']], x_test_scores,on='business_id')

attributes_df = pd.json_normalize(merged_df['attributes'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else {}))



merged_df['label_divided'] = merged_df['label'] / merged_df['review_count']
x_test = merged_df[['review_count', 'label_divided', 'is_open']]
x_test['price'] = attributes_df['RestaurantsPriceRange2'].replace('None', np.nan).astype(float)
average_price = x_train['price'].mean()
x_test['attributes'] = attributes_df.loc[:, attributes_df.eq('True').any()].count(axis=1)
x_test['price'].fillna(average_price, inplace=True)

x_test


C:\Users\danie\AppData\Local\Temp\ipykernel_7060\441653240.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test['price'] = attributes_df['RestaurantsPriceRange2'].replace('None', np.nan).astype(float)
C:\Users\danie\AppData\Local\Temp\ipykernel_7060\441653240.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test['attributes'] = attributes_df.loc[:, attributes_df.eq('True').any()].count(axis=1)
C:\Users\danie\AppData\Local\Temp\ipykernel_7060\441653240.py:15: FutureWarning: A value is trying to 

,review_count,label_divided,is_open,price,attributes
0,9,-0.110696,1,2.000000,10
1,6,-0.000291,1,1.948135,0
2,8,0.998930,1,3.000000,0
3,13,0.844799,0,1.948135,3
4,3,0.333033,0,3.000000,0
...,...,...,...,...,...
1319,4,-0.000277,0,2.000000,1
1320,3,0.998871,1,4.000000,0
1321,62,0.788955,1,1.948135,1
1322,9,0.554772,1,1.000000,6


In [50]:
from catboost import Pool

eval_data = Pool(x_train, y_train)


In [56]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV


# Create a CatBoost classifier object
classifier = CatBoostClassifier(iterations=1000,
                            per_float_feature_quantization=['1:border_count=1024'],
                            learning_rate=0.1,
                            depth=5,
                            loss_function='CrossEntropy',
                           task_type="GPU",
                           eval_metric='F1',
                           devices='0')

# Train the classifier on x_train and y_train
classifier.fit(x_train, y_train, eval_set = eval_data)





0:	learn: 0.7679714	test: 0.7679714	best: 0.7679714 (0)	total: 119ms	remaining: 1m 59s
1:	learn: 0.7737705	test: 0.7737705	best: 0.7737705 (1)	total: 207ms	remaining: 1m 43s
2:	learn: 0.7738307	test: 0.7738307	best: 0.7738307 (2)	total: 314ms	remaining: 1m 44s
3:	learn: 0.7721872	test: 0.7721872	best: 0.7738307 (2)	total: 425ms	remaining: 1m 45s
4:	learn: 0.7719971	test: 0.7719971	best: 0.7738307 (2)	total: 507ms	remaining: 1m 40s
5:	learn: 0.7731867	test: 0.7731867	best: 0.7738307 (2)	total: 596ms	remaining: 1m 38s
6:	learn: 0.7763352	test: 0.7763352	best: 0.7763352 (6)	total: 679ms	remaining: 1m 36s
7:	learn: 0.7779583	test: 0.7779583	best: 0.7779583 (7)	total: 763ms	remaining: 1m 34s
8:	learn: 0.7793705	test: 0.7793705	best: 0.7793705 (8)	total: 859ms	remaining: 1m 34s
9:	learn: 0.7794508	test: 0.7794508	best: 0.7794508 (9)	total: 958ms	remaining: 1m 34s
10:	learn: 0.7822155	test: 0.7822155	best: 0.7822155 (10)	total: 1.04s	remaining: 1m 33s
11:	learn: 0.7823201	test: 0.7823201	best

In [ ]:
# Define the parameter grid
param_grid = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.1, 0.3, 0.5],
    'depth': [5, 10, 15],
    'loss_function': ['Logloss', 'CrossEntropy']
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, cv=5)

# Fit the grid search to the training data
grid_search.fit(x_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

In [52]:
# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Best Parameters: {'depth': 5, 'iterations': 200, 'learning_rate': 0.1, 'loss_function': 'Logloss'}
Best Score: 0.9185531805050264


In [57]:
classifier.get_feature_importance()

array([24.3026364 , 63.36811506,  2.31542279,  1.68280894,  8.3310168 ])

In [37]:
x_submission = pd.read_csv('./kaggle/input/desafioutfpr2024/X_testToronto.csv')

# Create a new DataFrame with business_id and predictions
predictions = pd.DataFrame({'business_id': x_submission['business_id'], 'destaque': grid_search.best_estimator_.predict(x_test)})

# Export the DataFrame to a CSV file
predictions.to_csv('predictions6.csv', index=False)
